# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('../data/AES.csv')
df

,Passwords,ciphertext,key
0,2428031609,b'1092675125059b44fa88',ZDBQ7O9F
1,4531040045,b'1e26ffabe380c3d0c299',NO9EDSU0
2,almigrana1,b'2259ed1d208cca75004d',4XS7R110
3,quiero95,b'16194b66fbca0eee',BMZPZLBH
4,doitnow2,b'fab56cb16f5cd504',19RM3TPZ
...,...,...,...
199995,dime168,b'ee2b9cf4479a64',E68PIJK0
199996,!l0v3k3v!n,b'e56a59f153590322ed74',REKBYL0P
199997,9870tmf05,b'0faaf9b73bc5bc9940',3QJZBLJP
199998,CAMILA,b'a451626f19cb',SOMFMV7Q


In [3]:
for i in range(len(df['Passwords'])):
    if len(df['Passwords'][i]) > 20:
        df.drop(i, axis=0, inplace=True)

In [4]:
df = df.dropna()[:60000].reset_index(drop=True)

In [5]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,2428031609,b'1092675125059b44fa88',ZDBQ7O9F
1,4531040045,b'1e26ffabe380c3d0c299',NO9EDSU0
2,almigrana1,b'2259ed1d208cca75004d',4XS7R110
3,quiero95,b'16194b66fbca0eee',BMZPZLBH
4,doitnow2,b'fab56cb16f5cd504',19RM3TPZ
...,...,...,...
59995,ayp081204,b'4b6abcaaeb85c1d0c2',NO9EDSU0
59996,vtec1500,b'54d2300a24039a42',ZDBQ7O9F
59997,hothick17,b'5efdbaef26cbb19842',3QJZBLJP
59998,9778hcfg,b'de27181e3de976e2',SOMFMV7Q


In [6]:
def removeExtra(x):
    x = x[2:-1]
    return x

df['ciphertext'] = df['ciphertext'].apply(lambda x: removeExtra(x))
df

,Passwords,ciphertext,key
0,2428031609,1092675125059b44fa88,ZDBQ7O9F
1,4531040045,1e26ffabe380c3d0c299,NO9EDSU0
2,almigrana1,2259ed1d208cca75004d,4XS7R110
3,quiero95,16194b66fbca0eee,BMZPZLBH
4,doitnow2,fab56cb16f5cd504,19RM3TPZ
...,...,...,...
59995,ayp081204,4b6abcaaeb85c1d0c2,NO9EDSU0
59996,vtec1500,54d2300a24039a42,ZDBQ7O9F
59997,hothick17,5efdbaef26cbb19842,3QJZBLJP
59998,9778hcfg,de27181e3de976e2,SOMFMV7Q


## Preprocessing Data

In [7]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 
    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [8]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [9]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

In [10]:
preproc_code_sentences[0]

array([ 7, 16,  7, 15,  5, 13,  3, 19,  5, 12,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [11]:
len(code_tokenizer.word_index)+1

110

In [12]:
len(plaintext_tokenizer.word_index)+1

17

In [13]:
plaintext_tokenizer.word_index

{'e': 1,
 'f': 2,
 '0': 3,
 'b': 4,
 '2': 5,
 '5': 6,
 '9': 7,
 'c': 8,
 'a': 9,
 '3': 10,
 '4': 11,
 'd': 12,
 '7': 13,
 '6': 14,
 '1': 15,
 '8': 16}

In [14]:
preproc_code_sentences.shape

(60000, 20)

In [15]:
preproc_plaintext_sentences.shape

(60000, 40, 1)

# LSTM

In [16]:
from keras.layers import GRU, Input, Dense, TimeDistributed, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def lstm(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 128, return_sequences = True, name='Layer1')(x)
    seq = LSTM(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss=sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [17]:
tmp_x.shape

(60000, 40, 1)

In [18]:
lstm_model = lstm(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 40, 1)]           0         
_________________________________________________________________
Layer1 (LSTM)                (None, 40, 128)           66560     
_________________________________________________________________
Layer2 (LSTM)                (None, 40, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 40, 17)            2193      
Total params: 200,337
Trainable params: 200,337
Non-trainable params: 0
_________________________________________________________________


In [19]:
lstm_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
83/83 [==============================] - 22s 59ms/step - loss: 1.6396 - accuracy: 0.5466 - val_loss: 1.2547 - val_accuracy: 0.5936
Epoch 2/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2527 - accuracy: 0.5926 - val_loss: 1.2292 - val_accuracy: 0.5968
Epoch 3/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2377 - accuracy: 0.5940 - val_loss: 1.2241 - val_accuracy: 0.5966
Epoch 4/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2315 - accuracy: 0.5940 - val_loss: 1.2175 - val_accuracy: 0.5979
Epoch 5/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2362 - accuracy: 0.5952 - val_loss: 1.2149 - val_accuracy: 0.5983
Epoch 6/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2218 - accuracy: 0.5960 - val_loss: 1.2150 - val_accuracy: 0.5977
Epoch 7/15
83/83 [==============================] - 4s 46ms/step - loss: 1.2173 - accuracy: 0.5970 - val_loss: 1.2107 - val_accuracy: 0.5983
Epoch 8/15
8

In [26]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''
    return ''.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

def getPred(model, x, n):
    print('Predicted\t\tActual')
    for i in range(n):
        print(logits_to_text(model.predict([x[:n]])[i], plaintext_tokenizer), end='\t')
        print(df['Passwords'][i])

In [27]:
getPred(lstm_model, tmp_x, 5)

Predicted		Actual
ffb9dffff5c999bdfdd9	2428031609
feb9dffff5c999bdffdf	4531040045
ffeb9ffffcccccdddddd	almigrana1
fee4d0fffe999924	quiero95
feb9dffffe999924	doitnow2


# GRU

In [22]:
def gru(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 128, return_sequences = True, name='Layer1')(x)
    seq = GRU(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [23]:
gru_model = gru(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 40, 1)]           0         
_________________________________________________________________
Layer1 (GRU)                 (None, 40, 128)           50304     
_________________________________________________________________
Layer2 (GRU)                 (None, 40, 128)           99072     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 17)            2193      
Total params: 151,569
Trainable params: 151,569
Non-trainable params: 0
_________________________________________________________________


In [24]:
gru_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
83/83 [==============================] - 8s 47ms/step - loss: 1.9122 - accuracy: 0.5520 - val_loss: 1.2722 - val_accuracy: 0.5920
Epoch 2/15
83/83 [==============================] - 3s 41ms/step - loss: 1.2584 - accuracy: 0.5914 - val_loss: 1.2301 - val_accuracy: 0.5958
Epoch 3/15
83/83 [==============================] - 3s 41ms/step - loss: 1.2378 - accuracy: 0.5934 - val_loss: 1.2235 - val_accuracy: 0.5969
Epoch 4/15
83/83 [==============================] - 3s 40ms/step - loss: 1.2261 - accuracy: 0.5958 - val_loss: 1.2155 - val_accuracy: 0.5973
Epoch 5/15
83/83 [==============================] - 3s 40ms/step - loss: 1.2224 - accuracy: 0.5965 - val_loss: 1.2120 - val_accuracy: 0.5994
Epoch 6/15
83/83 [==============================] - 3s 39ms/step - loss: 1.2196 - accuracy: 0.5969 - val_loss: 1.2113 - val_accuracy: 0.5998
Epoch 7/15
83/83 [==============================] - 3s 41ms/step - loss: 1.2120 - accuracy: 0.5980 - val_loss: 1.2324 - val_accuracy: 0.5976
Epoch 8/15
83

In [28]:
getPred(gru_model, tmp_x, 5)

Predicted		Actual
f1ba555005b9999b888f	2428031609
f1ba850000b9999b888f	4531040045
ff1e0d000055555ddd8f	almigrana1
f403800000999998	quiero95
f770800000999998	doitnow2
